In [1]:
import simpy
import numpy as np

In [2]:
class JusticeSystem():
    def __init__(self, env) -> None:
        self.env = env

        # Initialisation
        self.active_cases = 10
        self.magistrate_cases = 10
        self.crown_cases = 10

    # The entire justice process
    def justice(self, env):
        while True:
            # If stack is below 0, make it zero
            if self.active_cases < 0: self.active_cases = 0
            if self.magistrate_cases < 0: self.magistrate_cases = 0
            if self.crown_cases < 0: self.crown_cases = 0

            # Influx of cases
            self.active_cases += max(0, np.random.normal(5, 0.4))

            # State at the beginning of the day
            print(f'At the beginning of day {env.now+1}, the number of active cases is {round(self.active_cases,2)}k cases, cases in magistrate backlog is {round(self.magistrate_cases,2)}k cases, cases in crown court backlog is {round(self.crown_cases,2)}k cases.')

            yield self.env.process(self.summoned())

            if self.magistrate_cases > 0:
                yield self.env.process(self.magistrate()) 
            
            if self.crown_cases > 0:
                yield self.env.process(self.crown())
            
            yield self.env.timeout(1)

    # The summoned process
    def summoned(self):
        # Cases dismissed
        temp = max(0, np.random.normal(2.5, 0.4))
        self.active_cases -= temp
        yield self.env.timeout(0)
        
        # Cases moved to magistrate
        temp = max(0, np.random.normal(2.5, 0.4))
        self.magistrate_cases += temp
        yield self.env.timeout(0) 

    # The magistrate process
    def magistrate(self):
        # Cases dismissed
        temp = max(0, np.random.normal(0.5, 0.4))            
        self.magistrate_cases -= temp
        self.active_cases -= temp
        yield self.env.timeout(0)    

        # Cases sentenced
        temp = max(0, np.random.normal(0.5, 0.4))
        self.magistrate_cases -= temp
        self.active_cases -= temp
        yield self.env.timeout(0)

        # Cases moved to crown court
        temp = max(0, np.random.normal(1.5, 0.4))
        self.magistrate_cases -= temp
        self.crown_cases += temp
        yield self.env.timeout(0)

    # the crown process
    def crown(self):
        # Cases dismissed
        temp = max(0, np.random.normal(0.5, 0.4))         
        self.crown_cases -= temp
        self.active_cases -= temp
        yield self.env.timeout(0)    

        # Cases sentenced
        temp = max(0, np.random.normal(1, 0.4))  
        self.crown_cases -= temp
        self.active_cases -= temp
        yield self.env.timeout(0)


In [3]:
def simulate(env):
    justice_system = JusticeSystem(env)
    env.process(justice_system.justice(env)) #Add process `summoned` to the simpy env
    
    yield env.timeout(0)

In [4]:
env = simpy.Environment()

env.process(simulate(env)) # Add process `simulate` to the simpy env
env.run(until=100)

At the beginning of day 1, the number of active cases is 14.54k cases, cases in magistrate backlog is 10k cases, cases in crown court backlog is 10k cases.
At the beginning of day 2, the number of active cases is 16.09k cases, cases in magistrate backlog is 11.18k cases, cases in crown court backlog is 10.38k cases.
At the beginning of day 3, the number of active cases is 15.33k cases, cases in magistrate backlog is 10.98k cases, cases in crown court backlog is 10.66k cases.
At the beginning of day 4, the number of active cases is 17.29k cases, cases in magistrate backlog is 12.01k cases, cases in crown court backlog is 10.7k cases.
At the beginning of day 5, the number of active cases is 17.65k cases, cases in magistrate backlog is 11.71k cases, cases in crown court backlog is 10.25k cases.
At the beginning of day 6, the number of active cases is 17.17k cases, cases in magistrate backlog is 10.66k cases, cases in crown court backlog is 10.0k cases.
At the beginning of day 7, the numbe